In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('hfl/rbt3')

tokenizer

BertTokenizerFast(name_or_path='hfl/rbt3', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [10]:
tokenizer.batch_encode_plus(
    ['明月装饰了你的窗子', '你装饰了别人的梦'],
    truncation=True,
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[101, 3209, 3299, 6163, 7652, 749, 872, 4638, 4970, 2094, 102], [101, 872, 6163, 7652, 749, 1166, 782, 4638, 3457, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [11]:
from datasets import load_from_disk

dataset = load_from_disk('./data/ChnSentiCorp')

#缩小数据规模，便于测试
dataset['train'] = dataset['train'].shuffle().select(range(2000))
dataset['test'] = dataset['test'].shuffle().select(range(100))

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 0
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
})

In [12]:
#第6章/编码
def f(data):
    return tokenizer.batch_encode_plus(data['text'], truncation=True)


dataset = dataset.map(f,
                      batched=True,
                      batch_size=1000,
                      num_proc=4,
                      remove_columns=['text'])

dataset

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['label'],
        num_rows: 0
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 100
    })
})

In [13]:
def f(data):
    return [len(i) <= 512 for i in data['input_ids']]


dataset = dataset.filter(f, batched=True, batch_size=1000, num_proc=4)

dataset

Filter (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1984
    })
    validation: Dataset({
        features: ['label'],
        num_rows: 0
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 99
    })
})

In [14]:
#第6章/加载模型
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('hfl/rbt3',
                                                           num_labels=2)

#统计模型参数量
sum([i.nelement() for i in model.parameters()]) / 10000

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


3847.8338

In [15]:
import torch

#模拟一批数据
data = {
    'input_ids': torch.ones(4, 10, dtype=torch.long),
    'token_type_ids': torch.ones(4, 10, dtype=torch.long),
    'attention_mask': torch.ones(4, 10, dtype=torch.long),
    'labels': torch.ones(4, dtype=torch.long)
}

#模型试算
out = model(**data)

out['loss'], out['logits'].shape

(tensor(0.5686, grad_fn=<NllLossBackward0>), torch.Size([4, 2]))

In [16]:
from datasets import load_metric

metric = load_metric('accuracy')

In [17]:
#第6章/定义评价函数
import numpy as np
from transformers.trainer_utils import EvalPrediction


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = logits.argmax(axis=1)
    return {'accuracy': (logits == labels).sum() / len(labels)}
    #return metric.compute(predictions=logits, references=labels)


#模拟输出
eval_pred = EvalPrediction(
    predictions=np.array([[0, 1], [2, 3], [4, 5], [6, 7]]),
    label_ids=np.array([1, 1, 0, 1]),
)

compute_metrics(eval_pred)

{'accuracy': 0.75}

In [18]:
#第6章/定义训练参数
from transformers import TrainingArguments

#定义训练参数
args = TrainingArguments(
    #定义临时数据保存路径
    output_dir='./output_dir',

    #定义测试执行的策略，可取值no、epoch、steps
    evaluation_strategy='steps',

    #定义每隔多少个step执行一次测试
    eval_steps=30,

    #定义模型保存策略，可取值no、epoch、steps
    save_strategy='steps',

    #定义每隔多少个step保存一次
    save_steps=30,

    #定义共训练几个轮次
    num_train_epochs=1,

    #定义学习率
    learning_rate=1e-4,

    #加入参数权重衰减，防止过拟合
    weight_decay=1e-2,

    #定义测试和训练时的批次大小
    per_device_eval_batch_size=16,
    per_device_train_batch_size=16,

    #定义是否要使用gpu训练
    no_cuda=False,
)

In [19]:
#第6章/定义训练器
from transformers import Trainer
from transformers.data.data_collator import DataCollatorWithPadding

#定义训练器
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer),
)

In [20]:
#第6章/测试数据整理函数
data_collator = DataCollatorWithPadding(tokenizer)

#获取一批数据
data = dataset['train'][:5]

#输出这些句子的长度
for i in data['input_ids']:
    print(len(i))

#调用数据整理函数
data = data_collator(data)

#查看整理后的数据
for k, v in data.items():
    print(k, v.shape)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


172
29
287
56
52
input_ids torch.Size([5, 287])
token_type_ids torch.Size([5, 287])
attention_mask torch.Size([5, 287])
labels torch.Size([5])


In [21]:
tokenizer.decode(data['input_ids'][0])

'[CLS] 拉 拉 1 很 早 就 买 过 了 ， 第 一 遍 读 是 看 热 闹 、 看 情 节 ； 等 到 再 次 阅 读 的 时 候 ， 已 将 书 中 所 罗 列 的 职 场 要 点 贴 上 标 记 。 其 实 本 书 意 在 总 结 作 者 在 职 场 中 的 经 验 以 及 技 巧 ， 但 高 明 之 处 在 于 她 将 所 有 的 教 条 融 汇 于 一 个 生 动 的 故 事 中 ， 并 塑 造 了 一 个 鲜 活 的 白 领 形 象 。 拉 拉 2 与 1 的 区 别 在 于 ， 1 中 故 事 情 节 更 加 跌 宕 ， 而 2 更 侧 重 于 职 场 经 验 的 总 结 。 总 之 ， 是 本 好 书 的 同 时 ， 可 以 教 会 我 们 不 少 东 西 。 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [22]:
#第6章/评价模型
trainer.evaluate()

{'eval_loss': 0.7114213705062866,
 'eval_accuracy': 0.5050505050505051,
 'eval_runtime': 4.4101,
 'eval_samples_per_second': 22.449,
 'eval_steps_per_second': 1.587}

In [23]:
#第6章/训练
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
30,No log,0.423485,0.828283
60,No log,0.384137,0.828283
90,No log,0.395534,0.868687
120,No log,0.392675,0.868687


TrainOutput(global_step=124, training_loss=0.3901086930305727, metrics={'train_runtime': 43.9915, 'train_samples_per_second': 45.1, 'train_steps_per_second': 2.819, 'total_flos': 68960599001280.0, 'train_loss': 0.3901086930305727, 'epoch': 1.0})

In [24]:
#第6章/从某个存档继续训练
trainer.train(resume_from_checkpoint='./output_dir/checkpoint-90')

Step,Training Loss,Validation Loss,Accuracy
120,No log,0.380434,0.878788


TrainOutput(global_step=124, training_loss=0.07492614561511625, metrics={'train_runtime': 6.8956, 'train_samples_per_second': 287.721, 'train_steps_per_second': 17.983, 'total_flos': 68960599001280.0, 'train_loss': 0.07492614561511625, 'epoch': 1.0})

In [34]:
#第6章/评价模型
trainer.evaluate()

{'eval_loss': 0.37950029969215393,
 'eval_accuracy': 0.8787878787878788,
 'eval_runtime': 0.4722,
 'eval_samples_per_second': 209.664,
 'eval_steps_per_second': 14.825,
 'epoch': 1.0}

In [35]:
#第6章/手动保存模型参数
trainer.save_model(output_dir='./output_dir/save_model')

In [37]:
from transformers import AutoModel

# 假设您知道模型的类型或配置
model = AutoModel.from_pretrained('./output_dir/save_model')


In [47]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

model.eval()

for i, data in enumerate(trainer.get_eval_dataloader()):
    break

# 移除data字典中的'labels'键
labels = data.pop('labels', None)

for k, v in data.items():
    data[k] = v.to(device)

num_labels = 3
# 假设classifier是一个额外的线性层，用于生成logits
classifier = torch.nn.Linear(model.config.hidden_size, num_labels)

out = model(**data)
predicted_labels = logits.argmax(dim=1)

for i in range(16):
    print(tokenizer.decode(data['input_ids'][i], skip_special_tokens=True))

    # 检查是否存在'labels'键
    if 'labels' in data:
        print('label=', data['labels'][i].item())
    else:
        print('label= Not available')

    print('predict=', predicted_labels[i].item())


为 了 白 彦 的 死 ， 伤 心 了 好 久 。 很 久 没 有 为 了 一 本 书 。 为 了 一 本 书 里 那 些 如 此 贴 近 生 活 的 文 字 而 感 动 伤 心 了 。 喜 欢 宁 默 的 潇 洒 ， 刚 开 始 的 时 候 还 在 为 他 们 两 个 在 面 对 爱 情 时 的 清 醒 而 不 解 。 最 后 直 到 白 彦 意 外 死 后 出 现 的 地 图 ， 宁 默 的 反 应 。 才 知 道 原 来 他 们 都 爱 彼 此 爱 的 那 么 深 。 。 。 。 想 起 了 宁 默 那 句 话 ， 她 跟 梁 葳 葳 说 她 宁 愿 输 给 她 ， 输 给 任 何 人 也 不 想 输 给 生 离 死 别 。 。 。 。
label= Not available
predict= 0
指 纹 、 蓝 牙 都 有 ， 又 是 独 显 ， 这 个 价 格 也 是 市 面 上 最 低 ， 很 实 惠 。 送 货 上 门 很 方 便 很 放 心 。
label= Not available
predict= 1
有 了 第 一 本 书 的 铺 垫 ， 读 第 二 本 的 时 候 开 始 进 入 状 态 。 基 本 上 第 二 本 就 围 绕 主 角 们 的 能 力 训 练 展 开 ， 故 事 的 主 要 发 生 场 地 设 置 在 美 洲 的 亚 马 逊 丛 林 。 心 里 一 直 疑 惑 这 和 西 藏 有 什 么 关 系 ， 不 过 大 概 看 完 全 书 才 能 知 道 内 里 的 线 索 。 其 中 描 述 了 很 多 热 带 雨 林 中 特 有 的 神 秘 动 植 物 以 及 一 些 生 存 技 巧 和 常 识 ， 受 益 匪 浅 。 能 够 想 像 出 要 写 这 样 一 部 书 ， 融 合 这 样 许 多 的 知 识 ， 作 者 需 要 花 费 多 少 心 血 来 搜 集 和 整 理 并 成 文 。
label= Not available
predict= 0
和 其 他 netbook 比 起 来, 这 款 机 器 还 是 有 点 贵 的. 另 外 大 陆 始 终 没 有 引 进 高 分 屏 的 版 本, 很 遗 憾. 升 级 内 存 需 要 拆 键 盘, 对 一 般 用 户 可 能 会 是 个 问 题.
label= Not ava